In [ ]:
!pip install datasets --upgrade
!pip install evaluate
!pip install jiwer
!pip install accelerate -U
!pip install transformers[torch]
!pip install -U 'wandb>=0.12.10'

In [2]:
MODEL_PROCESSOR_CHECKPOINT = "openai/whisper-small"
FEATURE_TOKEN_CHECKPOINT = "openai/whisper-small"
DATASET_NAME = "bond005/sberdevices_golos_10h_crowd"

## Huggingface login


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## WanDB login

In [ ]:
import wandb

wandb.login()

## Load tokenizer and feature extractor

In [ ]:
from transformers import WhisperFeatureExtractor, WhisperTokenizer

feature_extractor = WhisperFeatureExtractor.from_pretrained(FEATURE_TOKEN_CHECKPOINT)
tokenizer = WhisperTokenizer.from_pretrained(FEATURE_TOKEN_CHECKPOINT, language="russian", task="transcribe")

## Load and Preprocess unprocessed Dataset and preprocess it 
(you can skip it and download already preprocessed one)

In [ ]:
from datasets import load_dataset, Audio

common_voice = load_dataset(DATASET_NAME)
common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    batch["text"] = batch["transcription"] if batch["transcription"] else ""
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    batch["labels"] = tokenizer(batch["text"]).input_ids #or sentence
    return batch

In [ ]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=2)

In [ ]:
common_voice

## Upload dataset

In [ ]:
common_voice.push_to_hub('gggggggg123/sber-golos')

## Load preproccessed dataset

In [ ]:
from datasets import load_dataset

common_voice = load_dataset('gggggggg123/sber-golos')

## Load Processor and Model

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration

processor = WhisperProcessor.from_pretrained(MODEL_PROCESSOR_CHECKPOINT, language="russian", task="transcribe")
model = WhisperForConditionalGeneration.from_pretrained(MODEL_PROCESSOR_CHECKPOINT)

## DataCollator, Config, and Metrics

In [10]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


In [11]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [12]:
import evaluate

metric = evaluate.load("wer")

In [13]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    for index in reversed(range(len(label_str))):
      if label_str[index] == '':
        del label_str[index]
        del pred_str[index]

    print("="*len(label_str))
    print(label_str)
    print(pred_str)
    print("="*len(label_str))

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [14]:
metric.compute(predictions=["п р и в е т"], references=["п р ы в е т"])

0.16666666666666666

In [15]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

## Training

In [16]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-ru-golos",  # change to a repo name of your choice
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=300,
    max_steps=3000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["wandb"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
    hub_strategy="all_checkpoints",
    num_train_epochs=2
)


In [17]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()

## Upload to Huggingface

In [19]:
kwargs = {
    "dataset_tags": "mbond005/sberdevices_golos_10h_crowd",
    "dataset": "sberdevices_golos_10h",  # a 'pretty' name for the training dataset
    "dataset_args": "config: ru, split: test",
    "language": "ru",
    "model_name": "Whisper Small Ru - Slava Gosha",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}

In [ ]:
trainer.push_to_hub(**kwargs)

## Download trained model

In [ ]:
model_name = 'gggggggg123/whisper-small-ru-golos'
checkpoint_version = 'checkpoint-2000'
model = WhisperForConditionalGeneration.from_pretrained('gggggggg123/whisper-small-ru-golos')

In [ ]:
model